<a href="https://colab.research.google.com/github/TishweeMahjong/CS370_Group2/blob/Milestone3/CS370_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Milestone 1: Source Videos and Closed Captions

In [ ]:
!pip install pytube
!pip install youtube_transcript_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
from pytube import YouTube
from pytube import Playlist
from youtube_transcript_api import YouTubeTranscriptApi

playlist_url = 'https://www.youtube.com/playlist?list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL'
total_videos = 10

# directory to store downloaded videos and captions
video_directory = '/content/gdrive/Shareddrives/CS370_Group2/downloaded_videos'
os.makedirs(video_directory, exist_ok=True)

def download_video(video_url, video_directory):
    try:
        yt = YouTube(video_url)
        video_stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
        video_stream.download(output_path=video_directory)
        print(f"Downloaded: {yt.title}")
        return True
    except Exception as e:
        return False

def download_captions(video_id, video_directory):
    try:
        captions = YouTubeTranscriptApi.get_transcript(video_id)
        with open(os.path.join(video_directory, f"{video_id}_captions.json"), 'w', encoding='utf-8') as file:
            file.write(str(captions))
        print(f"Downloaded captions for video ID: {video_id}")
    except Exception as e:
        return False

# Fetch videos from the playlist
playlist = Playlist(playlist_url)

downloaded_videos = 0

for video_url in playlist.video_urls:
    if downloaded_videos >= total_videos:
        break

    if download_video(video_url, video_directory):
        video_id = video_url.split('v=')[1]
        download_captions(video_id, video_directory)
        downloaded_videos += 1

Downloaded: Vice President Kamala Harris: The 2023 60 Minutes Interview
Downloaded captions for video ID: G3Eup4mfJdA
Downloaded: Pink: The 60 Minutes Interview
Downloaded captions for video ID: 480OGItLZNo
Downloaded: Rich Paul: The 60 Minutes Interview
Downloaded captions for video ID: OA2Tj75T3fI
Downloaded: "Godfather of AI" Geoffrey Hinton: The 60 Minutes Interview
Downloaded captions for video ID: qrvK_KuIeJk
Downloaded: Gen. Mark Milley: The 60 Minutes Interview
Downloaded captions for video ID: oFVuQ0RP_As
Downloaded: Attorney General Merrick Garland: The 60 Minutes Interview
Downloaded captions for video ID: 4aPp8KX6EiU
Downloaded: Deion Sanders: The 2023 60 Minutes Interview
Downloaded captions for video ID: h8PSWeRLGXs
Downloaded: Volodymyr Zelenskyy: The 2023 60 Minutes Interview
Downloaded captions for video ID: Z8qC2tVkGeU
Downloaded: Denzel Washington | 60 Minutes Archive
Downloaded captions for video ID: Y9nM_9oBj2k
Downloaded: World Number 1 Pool Player Shane Van Boeni

## Milestone 2: Speech to Text

In [ ]:
!pip install -q pytube
!pip install -q git+https://github.com/openai/whisper.git

import os, re
import torch
from pathlib import Path
from pytube import YouTube

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
import whisper
from whisper.utils import get_writer

transcription_directory = '/content/gdrive/Shareddrives/CS370_Group2/transcriptions'
os.makedirs(transcription_directory, exist_ok=True)

def transcribe_video(video_path, output_file):
    try:
        model = whisper.load_model("base")
        result = model.transcribe(video_path)
        transcribed_text = result["text"]

        with open(output_file, 'w', encoding='utf-8') as text_file:
            text_file.write(transcribed_text)
        print(f"Transcribed video: {video_path}")

    except Exception as e:
        print(f"Error transcribing video: {video_path}, Error: {str(e)}")

for filename in os.listdir(video_directory):
    if filename.endswith(".mp4"):
        video_path = os.path.join(video_directory, filename)
        output_file = os.path.join(transcription_directory, f"{os.path.splitext(filename)[0]}.txt")
        transcribe_video(video_path, output_file)

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 85.1MiB/s]


Transcribed video: /content/gdrive/Shareddrives/CS370_Group2/downloaded_videos/Vice President Kamala Harris The 2023 60 Minutes Interview.mp4
Transcribed video: /content/gdrive/Shareddrives/CS370_Group2/downloaded_videos/Pink The 60 Minutes Interview.mp4
Transcribed video: /content/gdrive/Shareddrives/CS370_Group2/downloaded_videos/Rich Paul The 60 Minutes Interview.mp4
Transcribed video: /content/gdrive/Shareddrives/CS370_Group2/downloaded_videos/Godfather of AI Geoffrey Hinton The 60 Minutes Interview.mp4
Transcribed video: /content/gdrive/Shareddrives/CS370_Group2/downloaded_videos/Gen Mark Milley The 60 Minutes Interview.mp4
Transcribed video: /content/gdrive/Shareddrives/CS370_Group2/downloaded_videos/Attorney General Merrick Garland The 60 Minutes Interview.mp4
Transcribed video: /content/gdrive/Shareddrives/CS370_Group2/downloaded_videos/Deion Sanders The 2023 60 Minutes Interview.mp4
Transcribed video: /content/gdrive/Shareddrives/CS370_Group2/downloaded_videos/Volodymyr Zelens

## Milestone 3: Source Text to Target Text

In [ ]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyICU: filename=PyICU-2.12-cp310-cp310-linux_x86_64.whl size=1754546 sha256=72cb5b8e7c46f6963093bae78f09ffc0873e1d901188f68ec1df9b50e6b8b139
  Stored in directory: /root/.cache/pip/wheels/74/60/95/66d97ac2fdc8be8e526c4254047405fe77feaf064282d1ad07
Successfully built PyICU


In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import os

# Load pre-trained model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
translation_directory = '/content/gdrive/Shareddrives/CS370_Group2/translations'
if not os.path.exists(translation_directory):
    os.makedirs(translation_directory)

transcribed_files = [file for file in os.listdir(transcription_directory) if file.endswith('.txt')]

for file in transcribed_files:
    with open(os.path.join(transcription_directory, file), 'r', encoding='utf-8') as f:
        transcribed_text = f.read()

    chunk_size = 1000 # to translate whole text
    transcribed_chunks = [transcribed_text[i:i + chunk_size] for i in range(0, len(transcribed_text), chunk_size)]
    translated_chunks = []
    for chunk in transcribed_chunks:
        input_text = f"translate English to French: {chunk}"
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        translated_ids = model.generate(input_ids=input_ids, max_length=150, num_beams=4, early_stopping=True)
        translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
        translated_chunks.append(translated_text)

    translated_text = ' '.join(translated_chunks)

    translated_filename = f"{os.path.splitext(file)[0]}_translated.txt"
    with open(os.path.join(translation_directory, translated_filename), 'w', encoding='utf-8') as f:
        f.write(translated_text)

Translated and saved text for video: /content/gdrive/Shareddrives/CS370_Group2/translations/Vice President Kamala Harris The 2023 60 Minutes Interview_translated.txt
Translated and saved text for video: /content/gdrive/Shareddrives/CS370_Group2/translations/Pink The 60 Minutes Interview_translated.txt
Translated and saved text for video: /content/gdrive/Shareddrives/CS370_Group2/translations/Rich Paul The 60 Minutes Interview_translated.txt
Translated and saved text for video: /content/gdrive/Shareddrives/CS370_Group2/translations/Godfather of AI Geoffrey Hinton The 60 Minutes Interview_translated.txt
Translated and saved text for video: /content/gdrive/Shareddrives/CS370_Group2/translations/Gen Mark Milley The 60 Minutes Interview_translated.txt
Translated and saved text for video: /content/gdrive/Shareddrives/CS370_Group2/translations/Attorney General Merrick Garland The 60 Minutes Interview_translated.txt
Translated and saved text for video: /content/gdrive/Shareddrives/CS370_Group2